# Bulletin 17C Flood Frequency Analysis
**HydroLib** — Big Sandy River at Bruceton, TN (USGS 03606500)

This notebook demonstrates a complete EMA-based flood frequency analysis:
1. Download annual peak flows from USGS NWIS
2. Run Bulletin 17C Expected Moments Algorithm
3. Compute frequency table (RI 1.5–500 yr) with 90% confidence intervals
4. Compare station / weighted / regional skew options
5. Plot frequency curve
6. Save report and figures

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from hydrolib import USGSgage, Bulletin17C, Hydrograph, HydroReport
from hydrolib.core import kfactor_array
from hydrolib.freq_plot import plot_frequency_curve_streamlit

%matplotlib inline
plt.rcParams['figure.dpi'] = 110

## 1. Download Data

In [ ]:
SITE_NO = '03606500'  # Big Sandy River at Bruceton, TN

gage = USGSgage(SITE_NO)
gage.fetch_site_info()
print(f'Site:          {gage.site_name}')
print(f'Drainage area: {gage.drainage_area} sq mi')

peak_df = gage.download_peak_flow()
print(f'\nPeak flow record: {len(peak_df)} years')
print(f'Water years: {peak_df["water_year"].min()} – {peak_df["water_year"].max()}')
peak_df.head()

In [ ]:
daily_df = gage.download_daily_flow()
print(f'Daily flow record: {len(daily_df):,} days')

## 2. Hydrograph Plots

In [ ]:
fig1 = Hydrograph.plot_daily_timeseries(
    daily_df, site_name=gage.site_name, site_no=SITE_NO
)
plt.show()

In [ ]:
fig2 = Hydrograph.plot_summary_hydrograph(
    daily_df, site_name=gage.site_name, site_no=SITE_NO,
    percentiles=[10, 25, 50, 75, 90],
)
plt.show()

In [ ]:
fig3, fdc_stats = Hydrograph.plot_flow_duration_curve(
    daily_df, site_name=gage.site_name, site_no=SITE_NO
)
plt.show()
fdc_stats

## 3. Bulletin 17C EMA Analysis

In [ ]:
# Regional skew: nationwide B17C default (England et al. 2019)
REGIONAL_SKEW     = -0.302
REGIONAL_SKEW_SE  =  0.55
REGIONAL_SKEW_MSE =  REGIONAL_SKEW_SE ** 2  # 0.302

b17c = Bulletin17C(
    peak_flows=peak_df['peak_flow_cfs'].values,
    water_years=peak_df['water_year'].values.astype(int),
    regional_skew=REGIONAL_SKEW,
    regional_skew_mse=REGIONAL_SKEW_MSE,
)

results = b17c.run_analysis(method='ema')

print(f'Method:               {results.method.name}')
print(f'n peaks:              {results.n_peaks}')
print(f'n systematic:         {results.n_systematic}')
print(f'EMA converged:        {results.ema_converged}  ({results.ema_iterations} iterations)')
print()
print(f'μ  (log10 Q):         {results.mean_log:.4f}')
print(f'σ  (log10 Q):         {results.std_log:.4f}')
print(f'γ  station skew:      {results.skew_station:.4f}')
print(f'γ  weighted skew:     {results.skew_weighted:.4f}')
print(f'γ  skew used:         {results.skew_used:.4f}')
print()
print(f'MGBT threshold:       {results.low_outlier_threshold:,.0f} cfs')
print(f'Low outliers:         {results.n_low_outliers}')

## 4. Frequency Table (RI 1.5–500 years)

In [ ]:
AEP = np.array([0.667, 0.50, 0.20, 0.10, 0.04, 0.02, 0.01, 0.005, 0.002])
RI  = [1.5, 2, 5, 10, 25, 50, 100, 200, 500]

q_df  = b17c.compute_quantiles(aep=AEP)
ci_df = b17c.compute_confidence_limits(aep=AEP, confidence=0.90)

freq_table = pd.DataFrame({
    'Return Interval (yr)': RI,
    'AEP (%)':              [f'{p*100:.1f}%' for p in AEP],
    'Flow (cfs)':           q_df['flow_cfs'].round(0).astype(int),
    'Lower 90% CI':         ci_df['lower_5pct'].round(0).astype(int),
    'Upper 90% CI':         ci_df['upper_5pct'].round(0).astype(int),
}).set_index('Return Interval (yr)')

freq_table

## 5. Skew Comparison

In [ ]:
skew_options = {
    'Station Skew':  results.skew_station,
    'Weighted Skew': results.skew_weighted,
    'Regional Skew': REGIONAL_SKEW,
}

rows = []
for label, skew_val in skew_options.items():
    K = kfactor_array(skew_val, AEP)
    flows = 10 ** (results.mean_log + K * results.std_log)
    row = {f'RI={ri}yr': int(round(f)) for ri, f in zip(RI, flows)}
    row['γ'] = round(skew_val, 4)
    rows.append(pd.Series(row, name=label))

comparison = pd.DataFrame(rows)
# Put γ column first
cols = ['γ'] + [c for c in comparison.columns if c != 'γ']
comparison[cols]

## 6. Frequency Curve Plot

In [ ]:
# Single curve (weighted skew default)
fig = plot_frequency_curve_streamlit(
    b17c,
    site_name=gage.site_name,
    site_no=SITE_NO,
)
plt.show()

In [ ]:
# Multi-skew overlay
fig_multi = plot_frequency_curve_streamlit(
    b17c,
    site_name=gage.site_name,
    site_no=SITE_NO,
    skew_curves=skew_options,
)
plt.show()

## 7. Save Report and Figures

In [ ]:
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

report = HydroReport(gage, b17c)
figures = report.generate_all_figures(output_dir)
report.save_report(os.path.join(output_dir, 'flood_frequency_report.md'))

fig_multi.savefig(
    os.path.join(output_dir, 'frequency_curve_skew_comparison.png'),
    dpi=300, bbox_inches='tight'
)

freq_table.to_csv(os.path.join(output_dir, 'frequency_table.csv'))

print('Saved to', output_dir)
for f in sorted(os.listdir(output_dir)):
    print(f'  {f}')